In [1]:
import geopandas as gpd
from shapely.geometry import box
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from flask import Flask, request, jsonify, render_template
from zipfile import ZipFile, ZIP_DEFLATED
from io import TextIOWrapper
import csv, re
matplotlib.use('Agg')

In [6]:
geodata = gpd.read_file("locations.geojson")
us_states = gpd.read_file("shapes/cb_2018_us_state_20m.shp")

fig, ax = plt.subplots()

codes = []
for n in pd.Series(geodata["address"]).values:
    zipCodes = re.findall(r"[A-Z]{2}\s(\d{5})", n)
    try:
        if len(zipCodes[0]) != 0:
            codes.append(zipCodes[0])
    except:
            codes.append(None)
geodata['postal_code'] = codes
geodata = geodata[~geodata['postal_code'].isnull()]

if 'postal_code' in geodata.columns:
    geodata['postal_code'] = geodata['postal_code'].astype(int)

geodata = geodata[geodata["postal_code"] > 25000]
geodata = geodata[geodata["postal_code"] < 65000]

boundingBox = box(-95, 25, -60, 50)

geodata = geodata[geodata.intersects(boundingBox)]
us_states = us_states[us_states.intersects(boundingBox)].to_crs("EPSG:2022")

us_states.plot(ax=ax, color = "lightgray", figsize = (5, 5))
final = geodata.to_crs(us_states.crs)
final.plot(ax=ax, column = "postal_code", legend = True, cmap = "RdBu")

ax.set_axis_off()


plt.savefig('dashboard.svg', format='svg')
plt.close(fig)
return flask.Response(fig, header = {"Content-Type": "image/svg+xml"})
